In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scanpy as sc
import anndata
import scanorama

# sittings
sns.set_style("ticks",{'axes.grid' : True})

# Load the data
Read the h5ad file we saved in the preprocessing and integration

In [ ]:
# Load the saved AnnData object after Integrating
adata = sc.read_h5ad("characterized.h5ad")

adata

# Filter out the non myeloid cells

In [ ]:
# Filter to allow only the cancer-associated fibroblast cells
adata = adata[adata.obs.cell_type == 'myeloid']

# Plot the UMAP of the cancer-associated fibroblast cells
sc.pl.umap(adata, color = ['Sample'], frameon = False)

adata

# Remove the points far from the center

In [ ]:
# identify the top farest points from the centroid on the umap
umap_coords = adata.obsm["X_umap"]
umap_outlayer = adata.obs_names[np.argsort(np.linalg.norm(umap_coords - umap_coords.mean(axis=0), axis=1))[::-1][:15]] # only one outlayer

# Remove the umap outlayers
adata = adata[~adata.obs_names.isin(umap_outlayer)]

# Plot the UMAP of the cancer-associated fibroblast cells without the outlayers
sc.pl.umap(adata, color = ['Sample'], frameon = False)

adata

# Categorize the myeloid according to their subtype using marker genes

In [ ]:
# Identify the marker genes
marker_genes = {
"SPP1+macrophage":["SPP1", "MARCO"],
"C1QC+macrophage":["C1QA", "C1QB", "C1QC"],
"MDSC":["S100A8", "S100A9", "S100A12"],
"monocyte":["FCGR3A", "CDKN1C"],
# "cDC1":["CLEC9A", "XCR1"],
"cDC2":["CD1C", "FCER1A"],
"cDC3":["CCL22", "CCR7"],
"pDC":["PLD4"],
"proliferating":["MKI67"],
"KIT+mast":["KIT"],
"T":["CD3E"],
# "epithelial-like":["KRT18, KRT19"]
}

# Calculate cell type scores then visualize it
for subtype, markers in marker_genes.items():
    sc.tl.score_genes(adata, gene_list=markers, score_name=subtype) # Calculate cell type scores 
    sc.pl.umap(adata, color=[subtype, *markers], frameon = False) # Visualize cell type scores on UMAP

# Identify the celltype according to maximum score
adata.obs['cell_subtype'] = adata.obs[list(marker_genes.keys())].apply(lambda row: row.idxmax(), axis=1)

# Visualize the results on the UMAP of CAFs
sc.pl.umap(adata, color='cell_subtype', frameon = False)

# Save images for the cell subtype clustering

In [ ]:
# UMAP epithelial subtype alone
_, axs = plt.subplots(figsize=(10,8))
sc.pl.umap(adata, color = 'cell_subtype', frameon = False, save=f"Myeloid_subtype.png", ax=axs, size=100)

# UMAP of the epithelial and proliferative epithelial cells and all other categories
sc.pl.umap(adata, color = ['cell_type', 'cell_subtype', 'moffitt', 'treatment', 'stage', 'procedure', 'Sample'], frameon = False, save=f"Myeloid_all.png", size=50)

# Make dot plot and stacked violin plot

In [ ]:
# One dimensional list of all the marker genes
markers = [gene for genes in list(marker_genes.values()) for gene in genes]

# Visualize the marker genes in a dotplot
sc.pl.dotplot(adata, markers, groupby='cell_subtype', categories_order=list(marker_genes.keys()), save='Myeloid_dotplot.png')

# Visualize marker gene distributions in a dotplot
sc.pl.stacked_violin(adata, markers, groupby='cell_subtype', categories_order=list(marker_genes.keys()), save='Myeloid_stacked_violin.png');

# Save the AnnData object in h5ad format

In [ ]:
adata.write_h5ad('Myeloid_subtype.h5ad')